In [8]:
import os
import json
import langchain_google_genai as genai
import traceback
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2


In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
google_api = os.getenv("GOOGLE_API_KEY") 

In [5]:
model = genai.GoogleGenerativeAI(google_api_key=google_api, model="gemini-1.0-pro", temperature=0.3)

In [6]:
model

GoogleGenerativeAI(model='gemini-1.0-pro', google_api_key=SecretStr('**********'), temperature=0.3, client=genai.GenerativeModel(
    model_name='models/gemini-1.0-pro',
    generation_config={},
    safety_settings={},
    tools=None,
))

In [10]:
RESPONSE_JSON = {
    1:{
        "question": "multiple choice question",
        "options":{
            "a":"1st choice",
            "b": "2nd choice",
            "c": "3rd choice",
            "d": "4th choice"
        },
        "correct": "correct answer"
    },
    2:{
        "question": "multiple choice question",
        "options":{
            "a":"1st choice",
            "b": "2nd choice",
            "c": "3rd choice",
            "d": "4th choice"
        },
        "correct": "correct answer"
    },
    3:{
        "question": "multiple choice question",
        "options":{
            "a":"1st choice",
            "b": "2nd choice",
            "c": "3rd choice",
            "d": "4th choice"
        },
        "correct": "correct answer"
    }
}

In [46]:
template = """
Text : {text} 
You are an expert MCQ maker. Based on the above text , you are to create \
{number} MCQ questions for {subject} students in {tone} tone.
Avoid asking questions from the text that are related to date.
Make sure the questions are not repeated and also confirm the question along with answer \
from the given text. You are to provide output in RESPONE_JSON format. 
Make sure you create {number} questions.
### RESPONSE_JSON
{response_json}
"""

quiz_template = PromptTemplate(
    input_variables=['text','number', 'subject', 'tone', 'response_json'],
    template=template
)

quiz_chain = LLMChain(llm=model, prompt=quiz_template, output_key="quiz", verbose=True)

In [47]:
template2 = """
You are an expert grammar evaluator and question checker.
Given the MCQ quiz questions for {subject} students, you are to \
evaluate the {quiz} questions to check its complexity. Make sure the question \
consist less than 50 words and also determine whether the questions align with \
the capabilites of the students. Make sure to update the question and also maintain the \
provided tone.
## MCQ quiz questions
{quiz}
"""

evaluate_prompt = PromptTemplate(
    template = template2,
    input_variables=['subject', 'quiz']
)

evaluate_chain = LLMChain(llm=model, prompt=evaluate_prompt, output_key='review', verbose=True)

In [48]:
final_chain = SequentialChain(chains=[quiz_chain, evaluate_chain],
                              input_variables=['text', 'number', 'subject', 'tone', 'response_json'],
                              output_variables=['quiz', 'review'],verbose=True)


In [49]:
text = ""
with open("data.txt", "r") as file:
    text = file.read()

In [57]:
number = 5
subject = "sports ethusiast"
tone = "extremely hard"

In [58]:
with get_openai_callback() as cb:
    response = final_chain(
        {
            "text":text,
            "number":number,
            "subject": subject,
            "tone": tone,
            "response_json": json.dumps(RESPONSE_JSON)
        }
    )



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text : The Premier League is the highest level of the English football league system. Contested by 20 clubs, it operates on a system of promotion and relegation with the English Football League (EFL). Seasons typically run from August to May, with each team playing 38 matches: two against each other team, one home and one away.[1] Most games are played on Saturday and Sunday afternoons, with occasional weekday evening fixtures.[2]

The competition was founded as the FA Premier League on 20 February 1992 following the decision of First Division (top-tier league from 1888 until 1992) clubs to break away from the English Football League. However, teams may still be relegated to and promoted from the EFL Championship. The Premier League takes advantage of a Â£5 billion television rights deal, with Sky and BT Group securing the domestic rights to broadcast 128 and 32 games, respectively.[3

In [82]:
print(cb)

Tokens Used: 0
	Prompt Tokens: 0
	Completion Tokens: 0
Successful Requests: 0
Total Cost (USD): $0.0


In [59]:
response['quiz']

'{"1": {"question": "Which club holds the distinction of being the only one to complete a Premier League campaign without losing a single game?", "options": {"a": "Manchester United", "b": "Arsenal", "c": "Chelsea", "d": "Liverpool"}, "correct": "b"}, "2": {"question": "Which club won the Premier League title in the 2015-16 season, despite having odds of 5000/1 at the beginning of the season?", "options": {"a": "Leicester City", "b": "Tottenham Hotspur", "c": "Manchester City", "d": "Arsenal"}, "correct": "a"}, "3": {"question": "Which club has the highest revenue in the Premier League, according to the 2016-17 Deloitte Football Money League report?", "options": {"a": "Manchester United", "b": "Manchester City", "c": "Chelsea", "d": "Liverpool"}, "correct": "a"}, "4": {"question": "Which club has the second-highest number of European Cup/UEFA Champions League titles, with a record six English clubs having won fifteen European championships in total?", "options": {"a": "Manchester Unite

In [55]:
response

{'text': 'The Premier League is the highest level of the English football league system. Contested by 20 clubs, it operates on a system of promotion and relegation with the English Football League (EFL). Seasons typically run from August to May, with each team playing 38 matches: two against each other team, one home and one away.[1] Most games are played on Saturday and Sunday afternoons, with occasional weekday evening fixtures.[2]\n\nThe competition was founded as the FA Premier League on 20 February 1992 following the decision of First Division (top-tier league from 1888 until 1992) clubs to break away from the English Football League. However, teams may still be relegated to and promoted from the EFL Championship. The Premier League takes advantage of a Â£5 billion television rights deal, with Sky and BT Group securing the domestic rights to broadcast 128 and 32 games, respectively.[3][4] This deal will rise to Â£6.7 billion for the four seasons from 2025 to 2029.[5] The league is

In [61]:
json.loads(response['quiz'])

{'1': {'question': 'Which club holds the distinction of being the only one to complete a Premier League campaign without losing a single game?',
  'options': {'a': 'Manchester United',
   'b': 'Arsenal',
   'c': 'Chelsea',
   'd': 'Liverpool'},
  'correct': 'b'},
 '2': {'question': 'Which club won the Premier League title in the 2015-16 season, despite having odds of 5000/1 at the beginning of the season?',
  'options': {'a': 'Leicester City',
   'b': 'Tottenham Hotspur',
   'c': 'Manchester City',
   'd': 'Arsenal'},
  'correct': 'a'},
 '3': {'question': 'Which club has the highest revenue in the Premier League, according to the 2016-17 Deloitte Football Money League report?',
  'options': {'a': 'Manchester United',
   'b': 'Manchester City',
   'c': 'Chelsea',
   'd': 'Liverpool'},
  'correct': 'a'},
 '4': {'question': 'Which club has the second-highest number of European Cup/UEFA Champions League titles, with a record six English clubs having won fifteen European championships in to

In [63]:
response['review']

'**Question Complexity Evaluation:**\n\nAll questions are within the capabilities of sports enthusiast students. They are concise, with less than 50 words each, and focus on key facts and trivia related to the Premier League.\n\n**Question Alignment with Student Capabilities:**\n\nThe questions are appropriate for students who have a basic understanding of the Premier League and its history. They require knowledge of specific clubs, achievements, and statistics.\n\n**Updated Questions:**\n\nThe questions remain unchanged as they are already concise and clear.\n\n**Updated MCQ Quiz Questions:**\n\n{"1": {"question": "Which club went undefeated in a Premier League season?", "options": {"a": "Manchester United", "b": "Arsenal", "c": "Chelsea", "d": "Liverpool"}, "correct": "b"}, "2": {"question": "Who won the Premier League in 2015-16 despite long odds?", "options": {"a": "Leicester City", "b": "Tottenham Hotspur", "c": "Manchester City", "d": "Arsenal"}, "correct": "a"}, "3": {"question"

In [64]:
response['quiz']

'{"1": {"question": "Which club holds the distinction of being the only one to complete a Premier League campaign without losing a single game?", "options": {"a": "Manchester United", "b": "Arsenal", "c": "Chelsea", "d": "Liverpool"}, "correct": "b"}, "2": {"question": "Which club won the Premier League title in the 2015-16 season, despite having odds of 5000/1 at the beginning of the season?", "options": {"a": "Leicester City", "b": "Tottenham Hotspur", "c": "Manchester City", "d": "Arsenal"}, "correct": "a"}, "3": {"question": "Which club has the highest revenue in the Premier League, according to the 2016-17 Deloitte Football Money League report?", "options": {"a": "Manchester United", "b": "Manchester City", "c": "Chelsea", "d": "Liverpool"}, "correct": "a"}, "4": {"question": "Which club has the second-highest number of European Cup/UEFA Champions League titles, with a record six English clubs having won fifteen European championships in total?", "options": {"a": "Manchester Unite

In [69]:
type(response)

dict

In [70]:
response

{'text': 'The Premier League is the highest level of the English football league system. Contested by 20 clubs, it operates on a system of promotion and relegation with the English Football League (EFL). Seasons typically run from August to May, with each team playing 38 matches: two against each other team, one home and one away.[1] Most games are played on Saturday and Sunday afternoons, with occasional weekday evening fixtures.[2]\n\nThe competition was founded as the FA Premier League on 20 February 1992 following the decision of First Division (top-tier league from 1888 until 1992) clubs to break away from the English Football League. However, teams may still be relegated to and promoted from the EFL Championship. The Premier League takes advantage of a Â£5 billion television rights deal, with Sky and BT Group securing the domestic rights to broadcast 128 and 32 games, respectively.[3][4] This deal will rise to Â£6.7 billion for the four seasons from 2025 to 2029.[5] The league is

In [71]:
quiz = response.get("quiz")

In [72]:
quiz

'{"1": {"question": "Which club holds the distinction of being the only one to complete a Premier League campaign without losing a single game?", "options": {"a": "Manchester United", "b": "Arsenal", "c": "Chelsea", "d": "Liverpool"}, "correct": "b"}, "2": {"question": "Which club won the Premier League title in the 2015-16 season, despite having odds of 5000/1 at the beginning of the season?", "options": {"a": "Leicester City", "b": "Tottenham Hotspur", "c": "Manchester City", "d": "Arsenal"}, "correct": "a"}, "3": {"question": "Which club has the highest revenue in the Premier League, according to the 2016-17 Deloitte Football Money League report?", "options": {"a": "Manchester United", "b": "Manchester City", "c": "Chelsea", "d": "Liverpool"}, "correct": "a"}, "4": {"question": "Which club has the second-highest number of European Cup/UEFA Champions League titles, with a record six English clubs having won fifteen European championships in total?", "options": {"a": "Manchester Unite

In [73]:
type(quiz)

str

In [75]:
quiz = json.loads(quiz)
type(quiz)

dict

In [77]:
quiz

{'1': {'question': 'Which club holds the distinction of being the only one to complete a Premier League campaign without losing a single game?',
  'options': {'a': 'Manchester United',
   'b': 'Arsenal',
   'c': 'Chelsea',
   'd': 'Liverpool'},
  'correct': 'b'},
 '2': {'question': 'Which club won the Premier League title in the 2015-16 season, despite having odds of 5000/1 at the beginning of the season?',
  'options': {'a': 'Leicester City',
   'b': 'Tottenham Hotspur',
   'c': 'Manchester City',
   'd': 'Arsenal'},
  'correct': 'a'},
 '3': {'question': 'Which club has the highest revenue in the Premier League, according to the 2016-17 Deloitte Football Money League report?',
  'options': {'a': 'Manchester United',
   'b': 'Manchester City',
   'c': 'Chelsea',
   'd': 'Liverpool'},
  'correct': 'a'},
 '4': {'question': 'Which club has the second-highest number of European Cup/UEFA Champions League titles, with a record six English clubs having won fifteen European championships in to

In [76]:
for key, value in quiz.items():
    print(f"{key} = {value}")

1 = {'question': 'Which club holds the distinction of being the only one to complete a Premier League campaign without losing a single game?', 'options': {'a': 'Manchester United', 'b': 'Arsenal', 'c': 'Chelsea', 'd': 'Liverpool'}, 'correct': 'b'}
2 = {'question': 'Which club won the Premier League title in the 2015-16 season, despite having odds of 5000/1 at the beginning of the season?', 'options': {'a': 'Leicester City', 'b': 'Tottenham Hotspur', 'c': 'Manchester City', 'd': 'Arsenal'}, 'correct': 'a'}
3 = {'question': 'Which club has the highest revenue in the Premier League, according to the 2016-17 Deloitte Football Money League report?', 'options': {'a': 'Manchester United', 'b': 'Manchester City', 'c': 'Chelsea', 'd': 'Liverpool'}, 'correct': 'a'}
4 = {'question': 'Which club has the second-highest number of European Cup/UEFA Champions League titles, with a record six English clubs having won fifteen European championships in total?', 'options': {'a': 'Manchester United', 'b': 

In [78]:
formatted_data = []

for key, value in quiz.items():
    options = ' | '.join([f"{k}: {v}" for k, v in value['options'].items()])
    question_data = {
        'question_no': key,
        'question': value['question'],
        'options': options,
        'correct': value['correct']
    }
    formatted_data.append(question_data)


In [79]:
formatted_data

[{'question_no': '1',
  'question': 'Which club holds the distinction of being the only one to complete a Premier League campaign without losing a single game?',
  'options': 'a: Manchester United | b: Arsenal | c: Chelsea | d: Liverpool',
  'correct': 'b'},
 {'question_no': '2',
  'question': 'Which club won the Premier League title in the 2015-16 season, despite having odds of 5000/1 at the beginning of the season?',
  'options': 'a: Leicester City | b: Tottenham Hotspur | c: Manchester City | d: Arsenal',
  'correct': 'a'},
 {'question_no': '3',
  'question': 'Which club has the highest revenue in the Premier League, according to the 2016-17 Deloitte Football Money League report?',
  'options': 'a: Manchester United | b: Manchester City | c: Chelsea | d: Liverpool',
  'correct': 'a'},
 {'question_no': '4',
  'question': 'Which club has the second-highest number of European Cup/UEFA Champions League titles, with a record six English clubs having won fifteen European championships in 

In [80]:
import csv

In [81]:
csv_file_path = 'questions.csv'

# Write data to CSV file
with open(csv_file_path, 'w', newline='', encoding='utf-8') as csvfile:
    fieldnames = ['question_no', 'question', 'options', 'correct']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()
    for question_data in formatted_data:
        writer.writerow(question_data)

print(f"CSV file '{csv_file_path}' has been created successfully.")

CSV file 'questions.csv' has been created successfully.


In [2]:
from datetime import datetime
datetime.now().strftime("%d_%m_%Y_%H_%M_%S")

'08_04_2024_12_32_18'